### 전체 데이터 불러오기

In [4]:
import pandas as pd
import numpy as np
import sys
import os

# api.py가 있는 상위 경로 추가
sys.path.append(os.path.abspath('../../'))

In [2]:
# 청약매물 목록 정보
df_estate_list = pd.read_csv('청약 매물 목록.csv', encoding='cp949')

# 청약 신청자, 당첨자 정보
df_applicant_list = pd.read_csv('청약 지역별 신청자 목록.csv', encoding='cp949')
df_winner_list = pd.read_csv('청약 지역별 당첨자 목록.csv', encoding='cp949')

# 청약 당첨가점, 경쟁률 정보
df_estate_detail = pd.read_csv('청약매물_경쟁률_당첨가점_정보.csv', encoding='cp949')

In [207]:
df_estate_list.head()

,주택관리번호,공고번호,주택명,주택구분코드,주택구분코드명,주택상세구분코드,주택상세구분코드명,분양구분코드,분양구분코드명,공급지역코드,...,사업주체명_시행사,입주예정월,투기과열지구,조정대상지역,분양가상한제,정비사업,공공주택지구,대규모택지개발지구,수도권내민영공공주택지구,모집공고홈페이지주소
0,2025000043,2025000043,부천 JY 포에시아,1,APT,1,민영,0,분양주택,410,...,한국자산신탁(주),202504,N,N,N,N,N,N,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...
1,2025000032,2025000032,성남판교 산운마을9단지 대방노블랜드 아파트 우선 분양전환 후 잔여세대(15세대),1,APT,3,국민,0,분양주택,410,...,대방건설 주식회사,202504,N,N,N,N,N,Y,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...
2,2025000020,2025000020,e편한세상 제물포역 파크메종(조합원 취소분),1,APT,1,민영,0,분양주택,400,...,숭의3주택재개발정비사업조합,202607,N,N,N,Y,N,N,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...
3,2025000014,2025000014,평택청북 사랑으로 부영 2차 아파트 우선분양 전환 후 잔여세대,1,APT,1,민영,0,분양주택,410,...,(주)동광주택,202503,N,N,N,N,N,Y,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...
4,2025000013,2025000013,고양창릉지구 S-6블록 공공분양주택(본청약),1,APT,3,국민,0,분양주택,410,...,한국토지주택공사 경기북부지역본부,202712,N,N,Y,N,Y,Y,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...


In [208]:
# 매물 목록 API로 가져오기

from api import get_estate_list

df_estate_1 = get_estate_list('서울')
df_estate_2 = get_estate_list('경기')
df_estate_3 = get_estate_list('인천')

df_estate_list = pd.concat([df_estate_1, df_estate_2, df_estate_3])
df_estate_list.head()

,주택관리번호,공고번호,주택명,주택구분코드,주택구분코드명,주택상세구분코드,주택상세구분코드명,분양구분코드,분양구분코드명,공급지역코드,...,사업주체명_시행사,입주예정월,투기과열지구,조정대상지역,분양가상한제,정비사업,공공주택지구,대규모택지개발지구,수도권내민영공공주택지구,모집공고홈페이지주소
0,2025000001,2025000001,래미안 원페를라,01,APT,01,민영,0,분양주택,100,...,방배6구역주택재건축정비사업조합,202511,Y,Y,Y,Y,N,N,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...
1,2024000718,2024000718,더샵 퍼스트월드,01,APT,01,민영,0,분양주택,100,...,우리자산신탁 주식회사,202908,N,N,N,Y,N,N,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...
2,2024000692,2024000692,창경궁 롯데캐슬 시그니처,01,APT,01,민영,0,분양주택,100,...,삼선제5주택재개발정비사업조합,202704,N,N,N,Y,N,N,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...
3,2024000672,2024000672,아크로 리츠카운티,01,APT,01,민영,0,분양주택,100,...,방배삼익아파트 주택재건축정비사업조합,202710,Y,Y,Y,Y,N,N,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...
4,2024000688,2024000688,힐스테이트 등촌역,01,APT,01,민영,0,분양주택,100,...,등촌1구역주택재건축정비사업조합,202610,N,N,N,Y,N,N,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...


In [209]:
df_estate_list = df_estate_list.sort_values(by='모집공고일', ascending=False)
df_estate_list.to_csv('청약 매물 목록.csv', index=False, encoding='cp949')

### 병합 1) 청약 매물 목록 정보 + 신청자, 당첨자 정보

In [210]:
# 청약매물 목록 정보에서 청약접수시작월, 당첨자발표월 컬럼 추가

df_estate_list['청약접수시작월'] = pd.to_datetime(df_estate_list['청약접수시작일'], format='%Y-%m-%d').dt.strftime('%Y-%m')
df_estate_list['당첨자발표월'] = pd.to_datetime(df_estate_list['당첨자발표일'], format='%Y-%m-%d').dt.strftime('%Y-%m')
df_estate_list[['공급지역명', '청약접수시작월', '당첨자발표월']].head()

,공급지역명,청약접수시작월,당첨자발표월
0,경기,2025-03,2025-03
1,경기,2025-03,2025-03
0,인천,2025-03,2025-03
2,경기,2025-02,2025-02
4,경기,2025-02,2025-03


In [211]:
# 청약 연도별, 시도별 연령별 신청건수 API로 가져오기

from api import get_estate_applicant_list_total

df_applicant_1 = get_estate_applicant_list_total(100)  # 서울
df_applicant_2 = get_estate_applicant_list_total(400)  # 경기
df_applicant_3 = get_estate_applicant_list_total(410)  # 인천

df_applicant_list = pd.concat([df_applicant_1, df_applicant_2, df_applicant_3]).reset_index(drop=True)

In [212]:
df_applicant_list

,연월,시도,30대 이하 신청건수,40대 신청건수,50대 신청건수,60대 이상 신청건수
0,202501,서울,29368,19287,6625,3484
1,202412,서울,5649,1797,679,389
2,202411,서울,83524,31464,9972,5080
3,202410,서울,60856,39238,14854,7752
4,202409,서울,61378,47041,17843,8493
...,...,...,...,...,...,...
154,202007,경기,80691,30852,15995,8643
155,202006,경기,38162,12405,6449,3234
156,202005,경기,104189,47314,23834,12948
157,202004,경기,62318,19696,8580,4428


In [213]:
df_applicant_list.to_csv('청약 지역별 신청자 목록.csv', index=False, encoding='cp949')

In [214]:
# 청약 연도별, 시도별 연령별 당첨건수 API로 가져오기

from api import get_estate_winner_list_total

# 테스트 실행 예제
df_winner_1 = get_estate_winner_list_total(100)  
df_winner_2 = get_estate_winner_list_total(400) 
df_winner_3 = get_estate_winner_list_total(410) 

df_winner_list = pd.concat([df_winner_1, df_winner_2, df_winner_3]).reset_index(drop=True)

In [215]:
df_winner_list

,연월,시도,30대 이하 당첨건수,40대 당첨건수,50대 당첨건수,60대 이상 당첨건수
0,202501,서울,157,194,106,25
1,202412,서울,403,240,111,46
2,202411,서울,1526,931,353,157
3,202410,서울,262,241,107,32
4,202409,서울,204,132,70,18
...,...,...,...,...,...,...
154,202007,경기,5312,2441,1193,709
155,202006,경기,3162,1700,865,469
156,202005,경기,3423,2288,1056,477
157,202004,경기,3129,1327,527,295


In [216]:
df_winner_list.to_csv('청약 지역별 당첨자 목록.csv', index=False, encoding='cp949')

In [217]:
# 청약 신청자, 당첨자 정보 칼럼 가공

df_applicant_list['연월'] = pd.to_datetime(df_applicant_list['연월'], format='%Y%m', errors='coerce').dt.strftime('%Y-%m')
df_winner_list['연월'] = pd.to_datetime(df_winner_list['연월'], format='%Y%m', errors='coerce').dt.strftime('%Y-%m')

In [218]:
# 청약 목록 데이터프레임에 연월과 시도에 따라 신청자 데이터 병합

df_estate_list = pd.merge(
    df_estate_list, 
    df_applicant_list,
    left_on=['공급지역명', '청약접수시작월'], 
    right_on=['시도', '연월'],
    how='left',
)

df_estate_list.drop(columns=['청약접수시작월','연월', '시도'], inplace=True)

df_estate_list.head()

,주택관리번호,공고번호,주택명,주택구분코드,주택구분코드명,주택상세구분코드,주택상세구분코드명,분양구분코드,분양구분코드명,공급지역코드,...,정비사업,공공주택지구,대규모택지개발지구,수도권내민영공공주택지구,모집공고홈페이지주소,당첨자발표월,30대 이하 신청건수,40대 신청건수,50대 신청건수,60대 이상 신청건수
0,2025000043,2025000043,부천 JY 포에시아,01,APT,01,민영,0,분양주택,410,...,N,N,N,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,2025-03,NaN,NaN,NaN,NaN
1,2025000032,2025000032,성남판교 산운마을9단지 대방노블랜드 아파트 우선 분양전환 후 잔여세대(15세대),01,APT,03,국민,0,분양주택,410,...,N,N,Y,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,2025-03,NaN,NaN,NaN,NaN
2,2025000020,2025000020,e편한세상 제물포역 파크메종(조합원 취소분),01,APT,01,민영,0,분양주택,400,...,Y,N,N,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,2025-03,NaN,NaN,NaN,NaN
3,2025000014,2025000014,평택청북 사랑으로 부영 2차 아파트 우선분양 전환 후 잔여세대,01,APT,01,민영,0,분양주택,410,...,N,N,Y,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,2025-02,NaN,NaN,NaN,NaN
4,2025000013,2025000013,고양창릉지구 S-6블록 공공분양주택(본청약),01,APT,03,국민,0,분양주택,410,...,N,Y,Y,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,2025-03,NaN,NaN,NaN,NaN


In [219]:
# 청약 목록 데이터프레임에 연월과 시도에 따라 당첨자 데이터 병합

df_estate_list = pd.merge(
    df_estate_list, 
    df_winner_list,
    left_on=['공급지역명', '당첨자발표월'], 
    right_on=['시도', '연월'],
    how='left',
)

df_estate_list.drop(columns=['당첨자발표월','연월', '시도'], inplace=True)

df_estate_list

,주택관리번호,공고번호,주택명,주택구분코드,주택구분코드명,주택상세구분코드,주택상세구분코드명,분양구분코드,분양구분코드명,공급지역코드,...,수도권내민영공공주택지구,모집공고홈페이지주소,30대 이하 신청건수,40대 신청건수,50대 신청건수,60대 이상 신청건수,30대 이하 당첨건수,40대 당첨건수,50대 당첨건수,60대 이상 당첨건수
0,2025000043,2025000043,부천 JY 포에시아,01,APT,01,민영,0,분양주택,410,...,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025000032,2025000032,성남판교 산운마을9단지 대방노블랜드 아파트 우선 분양전환 후 잔여세대(15세대),01,APT,03,국민,0,분양주택,410,...,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025000020,2025000020,e편한세상 제물포역 파크메종(조합원 취소분),01,APT,01,민영,0,분양주택,400,...,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025000014,2025000014,평택청북 사랑으로 부영 2차 아파트 우선분양 전환 후 잔여세대,01,APT,01,민영,0,분양주택,410,...,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025000013,2025000013,고양창릉지구 S-6블록 공공분양주택(본청약),01,APT,03,국민,0,분양주택,410,...,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
848,2020000303,2020000303,수원호매실14단지 B-3BL,01,APT,03,국민,1,분양전환 가능임대,410,...,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,24191.0,9687.0,5176.0,2625.0,3129.0,1327.0,527.0,295.0
849,2020000302,2020000302,수원호매실15단지 B-7BL,01,APT,03,국민,1,분양전환 가능임대,410,...,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,24191.0,9687.0,5176.0,2625.0,3129.0,1327.0,527.0,295.0
850,2020000301,2020000301,수원호매실5단지 B-6BL,01,APT,03,국민,1,분양전환 가능임대,410,...,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,24191.0,9687.0,5176.0,2625.0,3129.0,1327.0,527.0,295.0
851,2020000304,2020000304,수원호매실13단지 A-4BL,01,APT,03,국민,1,분양전환 가능임대,410,...,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,24191.0,9687.0,5176.0,2625.0,3129.0,1327.0,527.0,295.0


### 병합 2) 청약 매물 목록 정보 + 당첨가점, 경쟁률 정보

In [ ]:
# 청약 경쟁률, 당첨가점 크롤링

from api import get_estate_detail

df_estate_list_ids = df_estate_list['공고번호'].unique().tolist()

df_estate_detail_total = pd.DataFrame()

for id in df_estate_list_ids:
    df_estate_detail = get_estate_detail(id)

    df_estate_detail_total = pd.concat([df_estate_detail_total, df_estate_detail])

In [187]:
df_estate_detail_total.to_csv('청약매물_경쟁률_당첨가점_정보.csv', index=False, encoding='cp949')

In [188]:
df_estate_detail_total_test = pd.read_csv('청약매물_경쟁률_당첨가점_정보.csv', encoding='cp949')

In [189]:
df_estate_detail_total_test

,주택관리번호,공고번호,주택형,공급세대수,순위,거주지역,접수건수,경쟁률,최저당첨가점,최고당첨가점,평균당첨가점
0,2025000043,2025000043,043.7207J,5,1순위,해당지역,0,(△5),NaN,NaN,NaN
1,2025000043,2025000043,043.7207J,5,1순위,기타지역,15,3.00,NaN,NaN,NaN
2,2025000043,2025000043,043.7207J,5,2순위,해당지역,3,-,NaN,NaN,NaN
3,2025000043,2025000043,043.7207J,5,2순위,기타지역,10,-,NaN,NaN,NaN
4,2025000043,2025000043,043.7207K,5,1순위,해당지역,1,(△4),NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
21226,2020000305,2020000305,059.8800A,3,1순위,기타경기,0,NaN,NaN,NaN,NaN
21227,2020000305,2020000305,059.8800A,3,1순위,기타지역,0,NaN,NaN,NaN,NaN
21228,2020000305,2020000305,059.8800A,3,2순위,해당지역,0,NaN,NaN,NaN,NaN
21229,2020000305,2020000305,059.8800A,3,2순위,기타경기,0,NaN,NaN,NaN,NaN


In [ ]:
# 청약 매물 목록 정보 + 당첨가점, 경쟁률 정보

df_estate_list = pd.merge(
    df_estate_list,
    df_estate_detail_total_test,
    on=['주택관리번호', '공고번호'], 
     how='inner',
).reset_index(drop=True)

In [191]:
df_estate_list

,주택관리번호,공고번호,주택명,주택구분코드,주택구분코드명,주택상세구분코드,주택상세구분코드명,분양구분코드,분양구분코드명,공급지역코드,...,모집공고홈페이지주소,주택형,공급세대수,순위,거주지역,접수건수,경쟁률,최저당첨가점,최고당첨가점,평균당첨가점
0,2025000043,2025000043,부천 JY 포에시아,1,APT,1,민영,0,분양주택,410,...,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,043.7207J,5,1순위,해당지역,0,(△5),NaN,NaN,NaN
1,2025000043,2025000043,부천 JY 포에시아,1,APT,1,민영,0,분양주택,410,...,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,043.7207J,5,1순위,기타지역,15,3.00,NaN,NaN,NaN
2,2025000043,2025000043,부천 JY 포에시아,1,APT,1,민영,0,분양주택,410,...,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,043.7207J,5,2순위,해당지역,3,-,NaN,NaN,NaN
3,2025000043,2025000043,부천 JY 포에시아,1,APT,1,민영,0,분양주택,410,...,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,043.7207J,5,2순위,기타지역,10,-,NaN,NaN,NaN
4,2025000043,2025000043,부천 JY 포에시아,1,APT,1,민영,0,분양주택,410,...,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,043.7207K,5,1순위,해당지역,1,(△4),NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21226,2020000305,2020000305,수원세류2단지 2BL,1,APT,3,국민,1,분양전환 가능임대,410,...,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,059.8800A,3,1순위,기타경기,0,NaN,NaN,NaN,NaN
21227,2020000305,2020000305,수원세류2단지 2BL,1,APT,3,국민,1,분양전환 가능임대,410,...,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,059.8800A,3,1순위,기타지역,0,NaN,NaN,NaN,NaN
21228,2020000305,2020000305,수원세류2단지 2BL,1,APT,3,국민,1,분양전환 가능임대,410,...,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,059.8800A,3,2순위,해당지역,0,NaN,NaN,NaN,NaN
21229,2020000305,2020000305,수원세류2단지 2BL,1,APT,3,국민,1,분양전환 가능임대,410,...,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,059.8800A,3,2순위,기타경기,0,NaN,NaN,NaN,NaN


### 3) 병합 데이터 저장

In [192]:
df_estate_list.to_csv('병합_청약매물_목록_정보.csv', index=False, encoding='cp949')

In [197]:
df_estate_list_merged = pd.read_csv('병합_청약매물_목록_정보.csv', encoding='cp949')
df_estate_list_merged

C:\Users\user\AppData\Local\Temp\ipykernel_19224\2474947920.py:1: DtypeWarning: Columns (36,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df_estate_list_merged = pd.read_csv('병합_청약매물_목록_정보.csv', encoding='cp949')


,주택관리번호,공고번호,주택명,주택구분코드,주택구분코드명,주택상세구분코드,주택상세구분코드명,분양구분코드,분양구분코드명,공급지역코드,...,모집공고홈페이지주소,주택형,공급세대수,순위,거주지역,접수건수,경쟁률,최저당첨가점,최고당첨가점,평균당첨가점
0,2025000043,2025000043,부천 JY 포에시아,1,APT,1,민영,0,분양주택,410,...,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,043.7207J,5,1순위,해당지역,0,(△5),NaN,NaN,NaN
1,2025000043,2025000043,부천 JY 포에시아,1,APT,1,민영,0,분양주택,410,...,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,043.7207J,5,1순위,기타지역,15,3.00,NaN,NaN,NaN
2,2025000043,2025000043,부천 JY 포에시아,1,APT,1,민영,0,분양주택,410,...,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,043.7207J,5,2순위,해당지역,3,-,NaN,NaN,NaN
3,2025000043,2025000043,부천 JY 포에시아,1,APT,1,민영,0,분양주택,410,...,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,043.7207J,5,2순위,기타지역,10,-,NaN,NaN,NaN
4,2025000043,2025000043,부천 JY 포에시아,1,APT,1,민영,0,분양주택,410,...,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,043.7207K,5,1순위,해당지역,1,(△4),NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21226,2020000305,2020000305,수원세류2단지 2BL,1,APT,3,국민,1,분양전환 가능임대,410,...,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,059.8800A,3.0,1순위,기타경기,0,NaN,NaN,NaN,NaN
21227,2020000305,2020000305,수원세류2단지 2BL,1,APT,3,국민,1,분양전환 가능임대,410,...,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,059.8800A,3.0,1순위,기타지역,0,NaN,NaN,NaN,NaN
21228,2020000305,2020000305,수원세류2단지 2BL,1,APT,3,국민,1,분양전환 가능임대,410,...,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,059.8800A,3.0,2순위,해당지역,0,NaN,NaN,NaN,NaN
21229,2020000305,2020000305,수원세류2단지 2BL,1,APT,3,국민,1,분양전환 가능임대,410,...,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,059.8800A,3.0,2순위,기타경기,0,NaN,NaN,NaN,NaN


### 4) 공급금액 크롤링 후 추가

In [198]:
import requests
from bs4 import BeautifulSoup
import time

In [199]:
# 청약 매물 공급금액 정보 가져오기

def get_estate_price(estate_id):
    query = f'houseManageNo={estate_id}&pblancNo={estate_id}'
    URL = f'https://www.applyhome.co.kr/ai/aia/selectAPTLttotPblancDetail.do?{query}'

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    response = requests.get(URL, headers=headers)
    response.content

    # BeautifulSoup으로 HTML 파싱
    soup = BeautifulSoup(response.content, 'lxml')

    # 공급금액 존재하는 테이블 찾기
    table = soup.findAll('table')[-2]

    # DataFrame 생성
    df = pd.read_html(str(table), converters={0:str, 1:str, 2:str})[0] # converters: 각 컬럼의 데이터를 문자열로 변환

    return df

df_estate_price = get_estate_price(2020000009)
df_estate_price

C:\Users\user\AppData\Local\Temp\ipykernel_19224\998599964.py:18: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  table = soup.findAll('table')[-2]
C:\Users\user\AppData\Local\Temp\ipykernel_19224\998599964.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table), converters={0:str, 1:str, 2:str})[0] # converters: 각 컬럼의 데이터를 문자열로 변환


,주택형,공급금액(최고가 기준),2순위 청약금
0,074.9100,45600,청약통장으로 청약(청약금 없음)
1,074.8900,45600,청약통장으로 청약(청약금 없음)


In [1]:
# 청약 매물 목록 데이터에 공급금액(최고가 기준) 컬럼 추가

# 청약 매물 목록 id 가져오기
estate_ids = df_estate_list_merged['공고번호'].unique().tolist()
estate_ids

# 빈 문자열 존재하여 제거
df_estate_list_merged['주택형'] = df_estate_list_merged['주택형'].str.strip()

for estate_id in estate_ids:
    try:
        # 청약 매물 공급금액 정보 가져오기
        df_estate_price = get_estate_price(estate_id)

        # 청약 매물 목록 데이터에 공급금액(최고가 기준) 컬럼 추가
        for index in range(len(df_estate_price)):
            estate_type = str(df_estate_price.loc[index, '주택형']).strip()
            estate_price = float(df_estate_price.loc[index, '공급금액(최고가 기준)']) * 10000

            mask = (df_estate_list_merged['공고번호'] == estate_id) & (df_estate_list_merged['주택형'] == estate_type)
            df_estate_list_merged.loc[mask, '공급금액(최고가 기준)'] = estate_price
    except:
        print(f'{estate_id} 데이터를 가져오는데 실패했습니다.')

df_estate_list_merged

NameError: name 'df_estate_list_merged' is not defined

In [201]:
df_estate_list_merged.to_csv('청약매물_공급금액 (서울, 경기, 인천).csv', index=False, encoding='cp949')

In [5]:
df_estate_list_merged = pd.read_csv('청약매물_공급금액 (서울, 경기, 인천).csv', encoding='cp949')
df_estate_list_merged.head()

C:\Users\user\AppData\Local\Temp\ipykernel_17464\3815782237.py:1: DtypeWarning: Columns (36,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df_estate_list_merged = pd.read_csv('청약매물_공급금액 (서울, 경기, 인천).csv', encoding='cp949')


,주택관리번호,공고번호,주택명,주택구분코드,주택구분코드명,주택상세구분코드,주택상세구분코드명,분양구분코드,분양구분코드명,공급지역코드,...,주택형,공급세대수,순위,거주지역,접수건수,경쟁률,최저당첨가점,최고당첨가점,평균당첨가점,공급금액(최고가 기준)
0,2025000043,2025000043,부천 JY 포에시아,1,APT,1,민영,0,분양주택,410,...,043.7207J,5,1순위,해당지역,0,(△5),NaN,NaN,NaN,377000000.0
1,2025000043,2025000043,부천 JY 포에시아,1,APT,1,민영,0,분양주택,410,...,043.7207J,5,1순위,기타지역,15,3.00,NaN,NaN,NaN,377000000.0
2,2025000043,2025000043,부천 JY 포에시아,1,APT,1,민영,0,분양주택,410,...,043.7207J,5,2순위,해당지역,3,-,NaN,NaN,NaN,377000000.0
3,2025000043,2025000043,부천 JY 포에시아,1,APT,1,민영,0,분양주택,410,...,043.7207J,5,2순위,기타지역,10,-,NaN,NaN,NaN,377000000.0
4,2025000043,2025000043,부천 JY 포에시아,1,APT,1,민영,0,분양주택,410,...,043.7207K,5,1순위,해당지역,1,(△4),NaN,NaN,NaN,377000000.0


### 테스트 데이터 가져오기
- 당첨자발표일이 현재일자를 넘기지 않은 매물 가져오기

In [17]:
import os
import sys
sys.path.append(os.path.abspath('../../'))
from api import get_future_estate_list, get_apartment_news, add_topic_keyword

# 당첨자 발표일 이전 청약 매물 가져오기
df_future_estate_list = get_future_estate_list()

# 주소로 위도, 경도, 법정동코드 가져오기

# 청약 매물 가져온거 뉴스 기사 크롤링해오기
df_news = get_apartment_news(df_future_estate_list)

# 뉴스기사 크롤링된거 토픽 모델링 후 피쳐 추가하기
df_future_estate_list = add_topic_keyword(df_future_estate_list, df_news)

1번 기사 - "41층에서 하버뷰를"용현 우방 아이유쉘 센트럴마린
2번 기사 - '안동용상하늘채리버스카이' 등 1205가구 분양예정[분양캘린더]
3번 기사 - 청년안심주택 보증금 238억 위험… 소형청약 경쟁률 160.9대 1 사상 최고...


In [18]:
pd.set_option('display.max_columns', None)

df_future_estate_list.head()

,주택관리번호,공고번호,주택명,주택구분코드,주택구분코드명,주택상세구분코드,주택상세구분코드명,분양구분코드,분양구분코드명,공급지역코드,공급지역명,공급위치우편번호,공급위치,공급규모,모집공고일,청약접수시작일,청약접수종료일,특별공급접수시작일,특별공급접수종료일,해당지역1순위접수시작일,해당지역1순위접수종료일,경기지역1순위접수시작일,경기지역1순위접수종료일,기타지역1순위접수시작일,기타지역1순위접수종료일,해당지역2순위접수시작일,해당지역2순위접수종료일,경기지역2순위접수시작일,경기지역2순위접수종료일,기타지역2순위접수시작일,기타지역2순위접수종료일,당첨자발표일,계약시작일,계약종료일,홈페이지주소,건설업체명_시공사,문의처,사업주체명_시행사,입주예정월,투기과열지구,조정대상지역,분양가상한제,정비사업,공공주택지구,대규모택지개발지구,수도권내민영공공주택지구,모집공고홈페이지주소,주택형,공급세대수,순위,거주지역,접수건수,경쟁률,최저당첨가점,최고당첨가점,평균당첨가점,토픽 1,토픽 2,토픽 3,토픽 4,토픽 5,토픽 6,토픽 7
0,2025000021,2025000021,용현 우방 아이유쉘 센트럴마린,01,APT,01,민영,0,분양주택,400,인천,22190,인천광역시 미추홀구 용현동 627-80번지,196,2025-03-14,2025-03-24,2025-03-26,2025-03-24,2025-03-24,2025-03-25,2025-03-25,None,None,2025-03-25,2025-03-25,2025-03-26,2025-03-26,None,None,2025-03-26,2025-03-26,2025-04-01,2025-04-14,2025-04-16,http://www.용현우방아이유쉘.com,"에스엠스틸(주) 건설부문, 경남기업(주)",0328593335,용현5동새한아파트재건축정비사업조합,202612,N,N,N,Y,N,N,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,059.9333,15,1순위,해당지역,0,NaN,NaN,NaN,NaN,0.179733,0.0011,0.0597,0.020833,0.081567,0.3104,0.346767
1,2025000021,2025000021,용현 우방 아이유쉘 센트럴마린,01,APT,01,민영,0,분양주택,400,인천,22190,인천광역시 미추홀구 용현동 627-80번지,196,2025-03-14,2025-03-24,2025-03-26,2025-03-24,2025-03-24,2025-03-25,2025-03-25,None,None,2025-03-25,2025-03-25,2025-03-26,2025-03-26,None,None,2025-03-26,2025-03-26,2025-04-01,2025-04-14,2025-04-16,http://www.용현우방아이유쉘.com,"에스엠스틸(주) 건설부문, 경남기업(주)",0328593335,용현5동새한아파트재건축정비사업조합,202612,N,N,N,Y,N,N,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,059.9333,15,1순위,기타지역,0,NaN,NaN,NaN,NaN,0.179733,0.0011,0.0597,0.020833,0.081567,0.3104,0.346767
2,2025000021,2025000021,용현 우방 아이유쉘 센트럴마린,01,APT,01,민영,0,분양주택,400,인천,22190,인천광역시 미추홀구 용현동 627-80번지,196,2025-03-14,2025-03-24,2025-03-26,2025-03-24,2025-03-24,2025-03-25,2025-03-25,None,None,2025-03-25,2025-03-25,2025-03-26,2025-03-26,None,None,2025-03-26,2025-03-26,2025-04-01,2025-04-14,2025-04-16,http://www.용현우방아이유쉘.com,"에스엠스틸(주) 건설부문, 경남기업(주)",0328593335,용현5동새한아파트재건축정비사업조합,202612,N,N,N,Y,N,N,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,059.9333,15,2순위,해당지역,0,NaN,NaN,NaN,NaN,0.179733,0.0011,0.0597,0.020833,0.081567,0.3104,0.346767
3,2025000021,2025000021,용현 우방 아이유쉘 센트럴마린,01,APT,01,민영,0,분양주택,400,인천,22190,인천광역시 미추홀구 용현동 627-80번지,196,2025-03-14,2025-03-24,2025-03-26,2025-03-24,2025-03-24,2025-03-25,2025-03-25,None,None,2025-03-25,2025-03-25,2025-03-26,2025-03-26,None,None,2025-03-26,2025-03-26,2025-04-01,2025-04-14,2025-04-16,http://www.용현우방아이유쉘.com,"에스엠스틸(주) 건설부문, 경남기업(주)",0328593335,용현5동새한아파트재건축정비사업조합,202612,N,N,N,Y,N,N,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,059.9333,15,2순위,기타지역,0,NaN,NaN,NaN,NaN,0.179733,0.0011,0.0597,0.020833,0.081567,0.3104,0.346767
4,2025000021,2025000021,용현 우방 아이유쉘 센트럴마린,01,APT,01,민영,0,분양주택,400,인천,22190,인천광역시 미추홀구 용현동 627-80번지,196,2025-03-14,2025-03-24,2025-03-26,2025-03-24,2025-03-24,2025-03-25,2025-03-25,None,None,2025-03-25,2025-03-25,2025-03-26,2025-03-26,None,None,2025-03-26,2025-03-26,2025-04-01,2025-04-14,2025-04-16,http://www.용현우방아이유쉘.com,"에스엠스틸(주) 건설부문, 경남기업(주)",0328593335,용현5동새한아파트재건축정비사업조합,202612,N,N,N,Y,N,N,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,073.4682,38,1순위,해당지역,0,NaN,NaN,NaN,NaN,0.179733,0.0011,0.0597,0.020833,0.081567,0.3104,0.346767


In [19]:
df_future_estate_list = get_future_estate_list()
df_future_estate_list.head()

,주택관리번호,공고번호,주택명,주택구분코드,주택구분코드명,주택상세구분코드,주택상세구분코드명,분양구분코드,분양구분코드명,공급지역코드,공급지역명,공급위치우편번호,공급위치,공급규모,모집공고일,청약접수시작일,청약접수종료일,특별공급접수시작일,특별공급접수종료일,해당지역1순위접수시작일,해당지역1순위접수종료일,경기지역1순위접수시작일,경기지역1순위접수종료일,기타지역1순위접수시작일,기타지역1순위접수종료일,해당지역2순위접수시작일,해당지역2순위접수종료일,경기지역2순위접수시작일,경기지역2순위접수종료일,기타지역2순위접수시작일,기타지역2순위접수종료일,당첨자발표일,계약시작일,계약종료일,홈페이지주소,건설업체명_시공사,문의처,사업주체명_시행사,입주예정월,투기과열지구,조정대상지역,분양가상한제,정비사업,공공주택지구,대규모택지개발지구,수도권내민영공공주택지구,모집공고홈페이지주소,주택형,공급세대수,순위,거주지역,접수건수,경쟁률,최저당첨가점,최고당첨가점,평균당첨가점
0,2025000021,2025000021,용현 우방 아이유쉘 센트럴마린,01,APT,01,민영,0,분양주택,400,인천,22190,인천광역시 미추홀구 용현동 627-80번지,196,2025-03-14,2025-03-24,2025-03-26,2025-03-24,2025-03-24,2025-03-25,2025-03-25,None,None,2025-03-25,2025-03-25,2025-03-26,2025-03-26,None,None,2025-03-26,2025-03-26,2025-04-01,2025-04-14,2025-04-16,http://www.용현우방아이유쉘.com,"에스엠스틸(주) 건설부문, 경남기업(주)",0328593335,용현5동새한아파트재건축정비사업조합,202612,N,N,N,Y,N,N,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,059.9333,15,1순위,해당지역,0,NaN,NaN,NaN,NaN
1,2025000021,2025000021,용현 우방 아이유쉘 센트럴마린,01,APT,01,민영,0,분양주택,400,인천,22190,인천광역시 미추홀구 용현동 627-80번지,196,2025-03-14,2025-03-24,2025-03-26,2025-03-24,2025-03-24,2025-03-25,2025-03-25,None,None,2025-03-25,2025-03-25,2025-03-26,2025-03-26,None,None,2025-03-26,2025-03-26,2025-04-01,2025-04-14,2025-04-16,http://www.용현우방아이유쉘.com,"에스엠스틸(주) 건설부문, 경남기업(주)",0328593335,용현5동새한아파트재건축정비사업조합,202612,N,N,N,Y,N,N,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,059.9333,15,1순위,기타지역,0,NaN,NaN,NaN,NaN
2,2025000021,2025000021,용현 우방 아이유쉘 센트럴마린,01,APT,01,민영,0,분양주택,400,인천,22190,인천광역시 미추홀구 용현동 627-80번지,196,2025-03-14,2025-03-24,2025-03-26,2025-03-24,2025-03-24,2025-03-25,2025-03-25,None,None,2025-03-25,2025-03-25,2025-03-26,2025-03-26,None,None,2025-03-26,2025-03-26,2025-04-01,2025-04-14,2025-04-16,http://www.용현우방아이유쉘.com,"에스엠스틸(주) 건설부문, 경남기업(주)",0328593335,용현5동새한아파트재건축정비사업조합,202612,N,N,N,Y,N,N,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,059.9333,15,2순위,해당지역,0,NaN,NaN,NaN,NaN
3,2025000021,2025000021,용현 우방 아이유쉘 센트럴마린,01,APT,01,민영,0,분양주택,400,인천,22190,인천광역시 미추홀구 용현동 627-80번지,196,2025-03-14,2025-03-24,2025-03-26,2025-03-24,2025-03-24,2025-03-25,2025-03-25,None,None,2025-03-25,2025-03-25,2025-03-26,2025-03-26,None,None,2025-03-26,2025-03-26,2025-04-01,2025-04-14,2025-04-16,http://www.용현우방아이유쉘.com,"에스엠스틸(주) 건설부문, 경남기업(주)",0328593335,용현5동새한아파트재건축정비사업조합,202612,N,N,N,Y,N,N,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,059.9333,15,2순위,기타지역,0,NaN,NaN,NaN,NaN
4,2025000021,2025000021,용현 우방 아이유쉘 센트럴마린,01,APT,01,민영,0,분양주택,400,인천,22190,인천광역시 미추홀구 용현동 627-80번지,196,2025-03-14,2025-03-24,2025-03-26,2025-03-24,2025-03-24,2025-03-25,2025-03-25,None,None,2025-03-25,2025-03-25,2025-03-26,2025-03-26,None,None,2025-03-26,2025-03-26,2025-04-01,2025-04-14,2025-04-16,http://www.용현우방아이유쉘.com,"에스엠스틸(주) 건설부문, 경남기업(주)",0328593335,용현5동새한아파트재건축정비사업조합,202612,N,N,N,Y,N,N,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,073.4682,38,1순위,해당지역,0,NaN,NaN,NaN,NaN


In [ ]:
# 주택명 정제

import pandas as pd
import re

def preprocess_apartname(df):

    # 특정 키워드 리스트 (정확한 일치 단어 제거)
    custom_stopwords = {}

    # 특정 단어를 포함하는 단어 제거 (예: '지구'가 포함된 단어 제거)
    remove_if_contains = ['우선' , '분양' ,'후', '잔여세대', '모집공고', '공급', '세대', '-', '전환', '블록', '공공']

    # ✅ 괄호와 괄호 안의 텍스트 모두 제거하는 함수
    def remove_parentheses(text):
        """괄호와 괄호 안의 텍스트 모두 제거"""
        return re.sub(r'\(.*?\)', '', text).strip()

    # ✅ 필터링 조건 함수
    def filter_conditions(word):
        """단어가 제거 대상인지 확인"""
        # if re.search(r'[A-Za-z]', word):  # 영어 포함 단어 제거
        #     return False
        if word in custom_stopwords:  # 특정 키워드 제거
            return False
        if any(sub in word for sub in remove_if_contains):  # 특정 단어 포함 단어 제거
            return False
        return True  # 모든 조건을 통과하면 유지

    # ✅ 정제 함수
    def clean_name(address):
        # 괄호와 괄호 안의 텍스트 제거
        address = remove_parentheses(address)

        words = address.split()  # 띄어쓰기 기준으로 분리
        cleaned_words = [word for word in words if filter_conditions(word)]  # 필터링 적용
        cleaned_address = ' '.join(cleaned_words)  # 정제된 단어들 다시 합치기

        # 쉼표가 있으면 첫 번째 단어만 반환
        if ',' in cleaned_address:
            return cleaned_address.split(',')[0]
        return cleaned_address  # 쉼표가 없으면 그대로 반환

    # ✅ 주소 정제 적용
    df['정제된주택명'] = df['공급지역명'] + ' ' + df['주택명'].apply(clean_name)

    return df

def preprocess_address(df):
    # 특정 키워드 리스트 (정확한 일치 단어 제거)
    custom_stopwords = {'내', '외'}

    # 특정 단어를 포함하는 단어 제거 (예: '지구'가 포함된 단어 제거)
    remove_if_contains = ['지구', '사업', '일원', '일대', '블록', '공동', '일반', '개발', '필지', '구역', '역세권', '블럭', '주상복합', '시티', '신도시', '종전', '부동산']

    # ✅ 괄호와 괄호 안의 텍스트 모두 제거하는 함수
    def remove_parentheses(text):
        """괄호와 괄호 안의 텍스트 모두 제거"""
        return re.sub(r'\(.*?\)', '', text).strip()

    # ✅ 필터링 조건 함수
    def filter_conditions(word):
        """단어가 제거 대상인지 확인"""
        if re.search(r'[A-Za-z]', word):  # 영어 포함 단어 제거
            return False
        if word in custom_stopwords:  # 특정 키워드 제거
            return False
        if any(sub in word for sub in remove_if_contains):  # 특정 단어 포함 단어 제거
            return False
        return True  # 모든 조건을 통과하면 유지

    # ✅ 정제 함수
    def clean_address(address):
        # 괄호와 괄호 안의 텍스트 제거
        address = remove_parentheses(address)

        words = address.split()  # 띄어쓰기 기준으로 분리
        cleaned_words = [word for word in words if filter_conditions(word)]  # 필터링 적용
        cleaned_address = ' '.join(cleaned_words)  # 정제된 단어들 다시 합치기

        # 쉼표가 있으면 첫 번째 단어만 반환
        if ',' in cleaned_address:
            return cleaned_address.split(',')[0]
        return cleaned_address  # 쉼표가 없으면 그대로 반환

    # ✅ 주소 정제 적용
    df['정제된주소'] = df['공급위치'].apply(clean_address)

    return df

def add_address_code(df_future_estate_list):
    df_future_estate_list_unique = df_future_estate_list.drop_duplicates(subset='공고번호', keep='first')
    df_future_estate_list_unique

    df_future_estate_list_unique = preprocess_apartname(df_future_estate_list_unique)
    df_future_estate_list_unique = preprocess_address(df_future_estate_list_unique)

    df_future_estate_list_unique['공급위치 by 주택명'] = df_future_estate_list_unique['정제된주택명'].apply(add_address_by_apartname)


,주택관리번호,공고번호,주택명,주택구분코드,주택구분코드명,주택상세구분코드,주택상세구분코드명,분양구분코드,분양구분코드명,공급지역코드,공급지역명,공급위치우편번호,공급위치,공급규모,모집공고일,청약접수시작일,청약접수종료일,특별공급접수시작일,특별공급접수종료일,해당지역1순위접수시작일,해당지역1순위접수종료일,경기지역1순위접수시작일,경기지역1순위접수종료일,기타지역1순위접수시작일,기타지역1순위접수종료일,해당지역2순위접수시작일,해당지역2순위접수종료일,경기지역2순위접수시작일,경기지역2순위접수종료일,기타지역2순위접수시작일,기타지역2순위접수종료일,당첨자발표일,계약시작일,계약종료일,홈페이지주소,건설업체명_시공사,문의처,사업주체명_시행사,입주예정월,투기과열지구,조정대상지역,분양가상한제,정비사업,공공주택지구,대규모택지개발지구,수도권내민영공공주택지구,모집공고홈페이지주소,주택형,공급세대수,순위,거주지역,접수건수,경쟁률,최저당첨가점,최고당첨가점,평균당첨가점
0,2025000021,2025000021,용현 우방 아이유쉘 센트럴마린,01,APT,01,민영,0,분양주택,400,인천,22190,인천광역시 미추홀구 용현동 627-80번지,196,2025-03-14,2025-03-24,2025-03-26,2025-03-24,2025-03-24,2025-03-25,2025-03-25,None,None,2025-03-25,2025-03-25,2025-03-26,2025-03-26,None,None,2025-03-26,2025-03-26,2025-04-01,2025-04-14,2025-04-16,http://www.용현우방아이유쉘.com,"에스엠스틸(주) 건설부문, 경남기업(주)",0328593335,용현5동새한아파트재건축정비사업조합,202612,N,N,N,Y,N,N,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,059.9333,15,1순위,해당지역,0,NaN,NaN,NaN,NaN


In [23]:
#.env 파일 로드
import requests
from dotenv import load_dotenv
load_dotenv()
kakao_api_key = os.getenv("kakao_api_key")

def add_address_by_apartname(apartname):
  kakao_api_url = "https://dapi.kakao.com/v2/local/search/keyword.json"
  headers = {"Authorization": f"KakaoAK {kakao_api_key}"}

  response = requests.get(kakao_api_url, headers=headers, params={"query": apartname})
  data = response.json()['documents']
  has_data = len(data) != 0
  
  if has_data:
    address = data[0]['address_name']
    return address
  
  return ''

In [24]:
df_future_estate_list_unique['공급위치 by 주택명'] = df_future_estate_list_unique['정제된주택명'].apply(add_address_by_apartname)

C:\Users\user\AppData\Local\Temp\ipykernel_15720\3738325461.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_future_estate_list_unique['공급위치 by 주택명'] = df_future_estate_list_unique['정제된주택명'].apply(add_address_by_apartname)


In [25]:
df_future_estate_list_unique

,주택관리번호,공고번호,주택명,주택구분코드,주택구분코드명,주택상세구분코드,주택상세구분코드명,분양구분코드,분양구분코드명,공급지역코드,공급지역명,공급위치우편번호,공급위치,공급규모,모집공고일,청약접수시작일,청약접수종료일,특별공급접수시작일,특별공급접수종료일,해당지역1순위접수시작일,해당지역1순위접수종료일,경기지역1순위접수시작일,경기지역1순위접수종료일,기타지역1순위접수시작일,기타지역1순위접수종료일,해당지역2순위접수시작일,해당지역2순위접수종료일,경기지역2순위접수시작일,경기지역2순위접수종료일,기타지역2순위접수시작일,기타지역2순위접수종료일,당첨자발표일,계약시작일,계약종료일,홈페이지주소,건설업체명_시공사,문의처,사업주체명_시행사,입주예정월,투기과열지구,조정대상지역,분양가상한제,정비사업,공공주택지구,대규모택지개발지구,수도권내민영공공주택지구,모집공고홈페이지주소,주택형,공급세대수,순위,거주지역,접수건수,경쟁률,최저당첨가점,최고당첨가점,평균당첨가점,정제된주택명,정제된주소,공급위치 by 주택명
0,2025000021,2025000021,용현 우방 아이유쉘 센트럴마린,01,APT,01,민영,0,분양주택,400,인천,22190,인천광역시 미추홀구 용현동 627-80번지,196,2025-03-14,2025-03-24,2025-03-26,2025-03-24,2025-03-24,2025-03-25,2025-03-25,None,None,2025-03-25,2025-03-25,2025-03-26,2025-03-26,None,None,2025-03-26,2025-03-26,2025-04-01,2025-04-14,2025-04-16,http://www.용현우방아이유쉘.com,"에스엠스틸(주) 건설부문, 경남기업(주)",0328593335,용현5동새한아파트재건축정비사업조합,202612,N,N,N,Y,N,N,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,059.9333,15,1순위,해당지역,0,NaN,NaN,NaN,NaN,인천 용현 우방 아이유쉘 센트럴마린,인천광역시 미추홀구 용현동 627-80번지,인천 미추홀구 용현동 627-80


In [26]:
def get_lat_lon_kakao(address):
    kakao_api_url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {kakao_api_key}"}

    try:
        response = requests.get(kakao_api_url, headers=headers, params={"query": address})
        # 요청 실패한경우
        if response.status_code != 200:
            return None
        
        result = response.json()
        data = result['documents']
        has_data = len(data) != 0

        # 주소 데이터 없는경우
        if not has_data:
            print(f'{address} - 주소 데이터가 없습니다.')
            return None
        
        result = data[0]['address']

        region_depth_3_h = result['region_3depth_h_name'] # 동
        region_depth_3 = result['region_3depth_name'] # 동

        # 동 데이터 없는 경우
        if region_depth_3_h == '' and region_depth_3 == '':
            print(f'{address} - 동 데이터가 없습니다.')
            return None

        return result
    except Exception as e:
        print(f"Error fetching data for address {address}: {e}")
        return None

def process_address_data(address, address_by_apartname, verbose=False):
    """카카오맵 API를 사용해 위도, 경도 및 행정정보를 추출하는 함수"""
    
    try:
        # 1️⃣ 카카오맵 API 사용해서 주소 데이터 가져오기
        data = get_lat_lon_kakao(address)

        # 2️⃣ 공급위치로 주소 못 얻는 경우, 주택명 기반으로 재시도
        if not data:
            data = get_lat_lon_kakao(address_by_apartname)

        # 3️⃣ 그래도 없는 경우 → NaN 처리
        if not data:
            if verbose:
                print(f"[주소 변환 실패] '{address}', '{address_by_apartname}' → NaN 값 반환")
            return None, None, None, None, None, None, None, None  # 8개 반환 보장

        # 4️⃣ 정상적인 데이터 추출
        lat = data.get('y', None)
        lon = data.get('x', None)
        h_code = data.get('h_code', None)
        b_code = data.get('b_code', None)
        region_depth_1 = data.get('region_1depth_name', None)  # 도
        region_depth_2 = data.get('region_2depth_name', None)  # 시,구
        region_depth_3_h = data.get('region_3depth_h_name', None)  # 행정동
        region_depth_3 = data.get('region_3depth_name', None)  # 법정동

        # 5️⃣ 필요할 때만 로그 출력 (verbose=True 설정 시)
        if verbose:
            print(f"[주소 변환 성공] '{address}' → {lat}, {lon}, {h_code}, {b_code}, {region_depth_1}, {region_depth_2}, {region_depth_3_h}, {region_depth_3}")

        return lat, lon, h_code, b_code, region_depth_1, region_depth_2, region_depth_3_h, region_depth_3
    
    except Exception as e:
        print(f"[오류 발생] {e} (address='{address}', address_by_apartname='{address_by_apartname}')")
        return None, None, None, None, None, None, None, None  # 오류 발생 시도 안전하게 8개 반환


In [27]:
df_future_estate_list_unique[['위도', '경도', '행정동코드', '법정동코드', '시도', '시군구', '읍면동1', '읍면동2']] = df_future_estate_list_unique.apply(
    lambda x: process_address_data(x['정제된주소'], x['공급위치 by 주택명']), axis=1, result_type='expand'
)

C:\Users\user\AppData\Local\Temp\ipykernel_15720\2615489161.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_future_estate_list_unique[['위도', '경도', '행정동코드', '법정동코드', '시도', '시군구', '읍면동1', '읍면동2']] = df_future_estate_list_unique.apply(
C:\Users\user\AppData\Local\Temp\ipykernel_15720\2615489161.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_future_estate_list_unique[['위도', '경도', '행정동코드', '법정동코드', '시도', '시군구', '읍면동1', '읍면동2']] = df_future_estate_list_unique.apply(
C:\Users\user\AppData\Local\Temp

In [30]:
df_future_estate_list_unique = df_future_estate_list_unique[['공고번호', '위도', '경도', '행정동코드', '법정동코드', '시도', '시군구', '읍면동1', '읍면동2']]

In [31]:
df_future_estate_list_unique

,공고번호,위도,경도,행정동코드,법정동코드,시도,시군구,읍면동1,읍면동2
0,2025000021,37.4512154061721,126.633728242814,2817757000,2817710200,인천,미추홀구,용현5동,용현동


In [32]:
df_future_estate_list = pd.merge(df_future_estate_list, df_future_estate_list_unique, how='inner')
df_future_estate_list

,주택관리번호,공고번호,주택명,주택구분코드,주택구분코드명,주택상세구분코드,주택상세구분코드명,분양구분코드,분양구분코드명,공급지역코드,공급지역명,공급위치우편번호,공급위치,공급규모,모집공고일,청약접수시작일,청약접수종료일,특별공급접수시작일,특별공급접수종료일,해당지역1순위접수시작일,해당지역1순위접수종료일,경기지역1순위접수시작일,경기지역1순위접수종료일,기타지역1순위접수시작일,기타지역1순위접수종료일,해당지역2순위접수시작일,해당지역2순위접수종료일,경기지역2순위접수시작일,경기지역2순위접수종료일,기타지역2순위접수시작일,기타지역2순위접수종료일,당첨자발표일,계약시작일,계약종료일,홈페이지주소,건설업체명_시공사,문의처,사업주체명_시행사,입주예정월,투기과열지구,조정대상지역,분양가상한제,정비사업,공공주택지구,대규모택지개발지구,수도권내민영공공주택지구,모집공고홈페이지주소,주택형,공급세대수,순위,거주지역,접수건수,경쟁률,최저당첨가점,최고당첨가점,평균당첨가점,위도,경도,행정동코드,법정동코드,시도,시군구,읍면동1,읍면동2
0,2025000021,2025000021,용현 우방 아이유쉘 센트럴마린,01,APT,01,민영,0,분양주택,400,인천,22190,인천광역시 미추홀구 용현동 627-80번지,196,2025-03-14,2025-03-24,2025-03-26,2025-03-24,2025-03-24,2025-03-25,2025-03-25,None,None,2025-03-25,2025-03-25,2025-03-26,2025-03-26,None,None,2025-03-26,2025-03-26,2025-04-01,2025-04-14,2025-04-16,http://www.용현우방아이유쉘.com,"에스엠스틸(주) 건설부문, 경남기업(주)",0328593335,용현5동새한아파트재건축정비사업조합,202612,N,N,N,Y,N,N,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,059.9333,15,1순위,해당지역,0,NaN,NaN,NaN,NaN,37.4512154061721,126.633728242814,2817757000,2817710200,인천,미추홀구,용현5동,용현동
1,2025000021,2025000021,용현 우방 아이유쉘 센트럴마린,01,APT,01,민영,0,분양주택,400,인천,22190,인천광역시 미추홀구 용현동 627-80번지,196,2025-03-14,2025-03-24,2025-03-26,2025-03-24,2025-03-24,2025-03-25,2025-03-25,None,None,2025-03-25,2025-03-25,2025-03-26,2025-03-26,None,None,2025-03-26,2025-03-26,2025-04-01,2025-04-14,2025-04-16,http://www.용현우방아이유쉘.com,"에스엠스틸(주) 건설부문, 경남기업(주)",0328593335,용현5동새한아파트재건축정비사업조합,202612,N,N,N,Y,N,N,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,059.9333,15,1순위,기타지역,0,NaN,NaN,NaN,NaN,37.4512154061721,126.633728242814,2817757000,2817710200,인천,미추홀구,용현5동,용현동
2,2025000021,2025000021,용현 우방 아이유쉘 센트럴마린,01,APT,01,민영,0,분양주택,400,인천,22190,인천광역시 미추홀구 용현동 627-80번지,196,2025-03-14,2025-03-24,2025-03-26,2025-03-24,2025-03-24,2025-03-25,2025-03-25,None,None,2025-03-25,2025-03-25,2025-03-26,2025-03-26,None,None,2025-03-26,2025-03-26,2025-04-01,2025-04-14,2025-04-16,http://www.용현우방아이유쉘.com,"에스엠스틸(주) 건설부문, 경남기업(주)",0328593335,용현5동새한아파트재건축정비사업조합,202612,N,N,N,Y,N,N,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,059.9333,15,2순위,해당지역,0,NaN,NaN,NaN,NaN,37.4512154061721,126.633728242814,2817757000,2817710200,인천,미추홀구,용현5동,용현동
3,2025000021,2025000021,용현 우방 아이유쉘 센트럴마린,01,APT,01,민영,0,분양주택,400,인천,22190,인천광역시 미추홀구 용현동 627-80번지,196,2025-03-14,2025-03-24,2025-03-26,2025-03-24,2025-03-24,2025-03-25,2025-03-25,None,None,2025-03-25,2025-03-25,2025-03-26,2025-03-26,None,None,2025-03-26,2025-03-26,2025-04-01,2025-04-14,2025-04-16,http://www.용현우방아이유쉘.com,"에스엠스틸(주) 건설부문, 경남기업(주)",0328593335,용현5동새한아파트재건축정비사업조합,202612,N,N,N,Y,N,N,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,059.9333,15,2순위,기타지역,0,NaN,NaN,NaN,NaN,37.4512154061721,126.633728242814,2817757000,2817710200,인천,미추홀구,용현5동,용현동
4,2025000021,2025000021,용현 우방 아이유쉘 센트럴마린,01,APT,01,민영,0,분양주택,400,인천,22190,인천광역시 미추홀구 용현동 627-80번지,196,2025-03-14,2025-03-24,2025-03-26,2025-03-24,2025-03-24,2025-03-25,2025-03-25,None,None,2025-03-25,2025-03-25,2025-03-26,2025-03-26,None,None,2025-03-26,2025-03-26,2025-04-01,2025-04-14,2025-04-16,http://www.용현우방아이유쉘.com,"에스엠스틸(주) 건설부문, 경남기업(주)",0328593335,용현5동새한아파트재건축정비사업조합,202612,N,N,N,Y,N,N,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,073.4682,38,1순위,해당지역,0,NaN,NaN,NaN,NaN,37.4512154061721,126.633728242814,2817757000,2817710200,인천,미추홀구,용현5동,용현동
5,2025000021,2025000021,용현 우방 아이유쉘 센트럴마린,01,APT,01,민영,0,분양주택,400,인천,22190,인천광역시 미추홀구 용현동 627-80번지,196,2025-03-14,2025-03-24,2025-03-26,2025-03-24,2025-03-24,2025-03-25,2025-03-25,None,None,2025-03-25,2025-03-25,2025-03-26,2025-03-26,None,None,2025-03-26,2025-03-26,2025-04-01,2025-04-14,2025-04-16,http://www.용현우방아이유쉘.com,"에스엠스틸(주) 건설부문, 경남기업(주)",0328593335,용현5동새한아파트재건축정비사업조합,202612,N,N,N,Y,N,N,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,073.4682,38,1순위,기타지역,0,NaN,NaN,NaN,NaN,37.4512154061721,126.633728242814,2817757000,2817710200,인천,미추홀구,용현5동,용현동
6,2025000021,2025000021,용현 우방 아이유쉘 센트럴마린,01,APT,01,민영,0,분양주택,400,인천,22190,인천광역시 미추홀구 용현동 627-80번지,196,2025-03-14,2025-03-24,2025-03-26,202